# Gaining perspective on stock prices

## Könyv 151. oldal
### Mivel ez részvényekről szól, inkább a StockInfo notebookba rakom a gyakorlatot.
# Translating SQL WHERE clauses
### Könyv 155. oldal

Pandas has the ability to connect to databases and send SQL statements to them.
This recipe will write pandas code that is equivalent to a SQL query that selects a certain subset
of the employee dataset.

In [1]:
import pandas as pd
employee = pd.read_csv('https://raw.githubusercontent.com/DatasRev/source-files/master/csv/employee.csv')
employee.head(7)

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22
5,5,SENIOR POLICE OFFICER,Houston Police Department-HPD,66614.0,Black or African American,Full Time,Male,Active,1984-11-26,2005-03-26
6,6,ENGINEER,Public Works & Engineering-PWE,71680.0,Asian/Pacific Islander,Full Time,Male,Active,2012-03-26,2012-03-26


In [2]:
employee.shape

(2000, 10)

In [3]:
employee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
UNIQUE_ID            2000 non-null int64
POSITION_TITLE       2000 non-null object
DEPARTMENT           2000 non-null object
BASE_SALARY          1886 non-null float64
RACE                 1965 non-null object
EMPLOYMENT_TYPE      2000 non-null object
GENDER               2000 non-null object
EMPLOYMENT_STATUS    2000 non-null object
HIRE_DATE            2000 non-null object
JOB_DATE             1997 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 156.3+ KB


In [4]:
employee.describe()

,UNIQUE_ID,BASE_SALARY
count,2000.000000,1886.000000
mean,999.500000,55767.931601
std,577.494589,21693.706679
min,0.000000,24960.000000
25%,499.750000,40170.000000
50%,999.500000,54461.000000
75%,1499.250000,66614.000000
max,1999.000000,275000.000000


In [5]:
employee.DEPARTMENT.value_counts().head()

Houston Police Department-HPD     638
Houston Fire Department (HFD)     384
Public Works & Engineering-PWE    343
Health & Human Services           110
Houston Airport System (HAS)      106
Name: DEPARTMENT, dtype: int64

In [6]:
employee.GENDER.value_counts()

Male      1397
Female     603
Name: GENDER, dtype: int64

In [7]:
employee.BASE_SALARY.describe().astype(int)

count      1886
mean      55767
std       21693
min       24960
25%       40170
50%       54461
75%       66614
max      275000
Name: BASE_SALARY, dtype: int32

Write a single statement for each of the criteria. Use the isin method to test
equality to one of many values:

The isin Series method is equivalent to the SQL IN operator and accepts a list of all
possible values that you would like to keep. It is possible to use a series of OR conditions to
replicate this expression but it would not be as efficient or idiomatic.

In [8]:
depts = ['Houston Police Department-HPD',
'Houston Fire Department (HFD)']
criteria_dept = employee.DEPARTMENT.isin(depts)
criteria_gender = employee.GENDER == 'Female'
criteria_sal = (employee.BASE_SALARY >= 80000) & \
(employee.BASE_SALARY <= 120000)

In [9]:
criteria_final = (criteria_dept & criteria_gender & criteria_sal)

In [10]:
# Use boolean indexing to select only the rows that meet the final criteria:
select_columns = ['UNIQUE_ID', 'DEPARTMENT', 'GENDER', 'BASE_SALARY']
employee.loc[criteria_final, select_columns].head()

,UNIQUE_ID,DEPARTMENT,GENDER,BASE_SALARY
61,61,Houston Fire Department (HFD),Female,96668.0
136,136,Houston Police Department-HPD,Female,81239.0
367,367,Houston Police Department-HPD,Female,86534.0
474,474,Houston Police Department-HPD,Female,91181.0
513,513,Houston Police Department-HPD,Female,81239.0


There's more...
For many operations, pandas has multiple ways to do the same thing. In the preceding
recipe, the criteria for salary uses two separate boolean expressions. Similarly to SQL, Series
have a between method, with the salary criteria equivalently written as follows:

criteria_sal = employee.BASE_SALARY.between(80000, 120000)

Another useful application of isin is to provide a sequence of values automatically
generated by some other pandas statements. This would avoid any manual investigating to
find the exact string names to store in a list. Conversely, let's try to exclude the rows from
the top five most frequently occurring departments:

In [11]:
top_5_depts = employee.DEPARTMENT.value_counts().index[:5]
criteria = ~employee.DEPARTMENT.isin(top_5_depts)
employee[criteria]

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22
18,18,MAINTENANCE MECHANIC III,General Services Department,40581.0,Hispanic/Latino,Full Time,Male,Active,2008-12-29,2008-12-29
32,32,SENIOR ACCOUNTANT,Finance,46963.0,Black or African American,Full Time,Male,Active,1991-02-11,2016-02-13
37,37,ASSISTANT SHOP MANAGER,Fleet Management Department,51438.0,White,Full Time,Male,Active,1995-11-30,2016-03-12
57,57,SEMI-SKILLED LABORER,Solid Waste Management,27622.0,Black or African American,Full Time,Male,Active,2015-08-31,2015-08-31
71,71,COUNCIL INTERN (EXECUTIVE LEVEL),City Council,NaN,Hispanic/Latino,Part Time 30,Male,Active,2015-06-08,2016-02-13
73,73,ADMINISTRATIVE SPECIALIST,Human Resources Dept.,55939.0,Black or African American,Full Time,Female,Active,2011-12-19,2013-11-23
74,74,SENIOR SIDELOADER OPERATOR,Solid Waste Management,38459.0,Black or African American,Full Time,Male,Active,2004-09-07,2005-07-09


# Determining the normality of stock market returns

# Improving readability of boolean indexing with the query method

# Könyv 165. oldal
Boolean indexing is not necessarily the most pleasant syntax to read or write, especially
when using a single line to write a complex filter. Pandas has an alternative string-based
syntax through the DataFrame query method that can provide more clarity.
The query DataFrame method is experimental and not as capable as
boolean indexing and should not be used for production code.

In [12]:
employee = pd.read_csv('https://raw.githubusercontent.com/DatasRev/source-files/master/csv/employee.csv')
employee.head(7)

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22
5,5,SENIOR POLICE OFFICER,Houston Police Department-HPD,66614.0,Black or African American,Full Time,Male,Active,1984-11-26,2005-03-26
6,6,ENGINEER,Public Works & Engineering-PWE,71680.0,Asian/Pacific Islander,Full Time,Male,Active,2012-03-26,2012-03-26


In [13]:
depts = ['Houston Police Department-HPD', 'Houston Fire Department (HFD)']
select_columns = ['UNIQUE_ID', 'DEPARTMENT', 'GENDER', 'BASE_SALARY']

In [14]:
# Build the query string and execute the method:
qs = "DEPARTMENT in @depts " \
"and GENDER == 'Female' " \
"and 80000 <= BASE_SALARY <= 120000"
emp_filtered = employee.query(qs)
emp_filtered[select_columns].head()

,UNIQUE_ID,DEPARTMENT,GENDER,BASE_SALARY
61,61,Houston Fire Department (HFD),Female,96668.0
136,136,Houston Police Department-HPD,Female,81239.0
367,367,Houston Police Department-HPD,Female,86534.0
474,474,Houston Police Department-HPD,Female,91181.0
513,513,Houston Police Department-HPD,Female,81239.0


Strings passed to the query method are going to look more like plain English than normal
pandas code. It is possible to reference Python variables using the at symbol (@) as with
depts. All DataFrame column names are available in the query namespace by simply
referencing their name without inner quotes. If a string is needed, such as Female, inner
quotes will need to wrap it.
Another nice feature of the query syntax is the ability to write a double inequality in a
single expression and its ability to understand the verbose logical operators and, or, and
not instead of their bitwise equivalents as with boolean indexing.

Instead of manually typing in a list of department names, we could have programmatically
created it. For instance, if we wanted to find all the female employees that were not a
member of the top 10 departments by frequency, we can run the following code:

In [15]:
top10_depts = employee.DEPARTMENT.value_counts() \
.index[:10].tolist()
qs = "DEPARTMENT not in @top10_depts and GENDER == 'Female'"
employee_filtered2 = employee.query(qs)
employee_filtered2.head()

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13
73,73,ADMINISTRATIVE SPECIALIST,Human Resources Dept.,55939.0,Black or African American,Full Time,Female,Active,2011-12-19,2013-11-23
96,96,ASSISTANT CITY CONTROLLER III,City Controller's Office,59077.0,Asian/Pacific Islander,Full Time,Female,Active,2013-06-10,2013-06-10
117,117,SENIOR ASSISTANT CITY ATTORNEY I,Legal Department,90957.0,Black or African American,Full Time,Female,Active,1998-03-20,2012-07-21
146,146,SENIOR STAFF ANALYST,Houston Information Tech Svcs,74951.0,White,Full Time,Female,Active,2014-03-17,2014-03-17
